# This is a sample Jupyter Notebook

Below is an example of a code cell. 
Put your cursor into the cell and press Shift+Enter to execute it and select the next one, or click 'Run Cell' button.

Press Double Shift to search everywhere for classes, files, tool windows, actions, and settings.

To learn more about Jupyter Notebooks in PyCharm, see [help](https://www.jetbrains.com/help/pycharm/ipython-notebook-support.html).
For an overview of PyCharm, go to Help -> Learn IDE features or refer to [our documentation](https://www.jetbrains.com/help/pycharm/getting-started.html).

1) Config + Seed +  Device

In [10]:
import os, time, json, random
import numpy as np

import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.utils.data import DataLoader, WeightedRandomSampler
from torchvision import datasets, transforms, models

from sklearn.metrics import accuracy_score, confusion_matrix

# -----------------------------
# PATHS
# -----------------------------
DATASET_ROOT = r".\datasetKaggleFER2013"   # train/ test klasörleri içinde
SAVE_DIR = "./runs_cnn"
os.makedirs(SAVE_DIR, exist_ok=True)

# -----------------------------
# TARGET
# -----------------------------
EPOCHS = 100               # 40 -> 60/80 genelde 0.70'e iter
BATCH  = 256              # VRAM yetmezse 128
IMG_SIZE = 224
UNFREEZE_EPOCH = 3        # 5 -> 3 (daha erken fine-tune)

# imbalance kontrol
USE_SAMPLER = True
USE_CLASS_WEIGHTS = False # sampler açıksa çoğu zaman kapatılmalı

LABEL_SMOOTH = 0.05       # 0.1 -> 0.05 (bazen accuracyyi yükseltir)
NUM_WORKERS = 2           # Windows: sorun olursa 0 yapılmalı

# Karcı alpha aralığı
KARCI_ALPHA = 1.35        # 0.8-1.8 arası, genelde 1.2-1.5 iyi
KARCI_CLIP  = 0.10
KARCI_EPS   = 1e-3

# cudnn speed
torch.backends.cudnn.benchmark = True

# seed
SEED = 42
random.seed(SEED); np.random.seed(SEED); torch.manual_seed(SEED)
if torch.cuda.is_available():
    torch.cuda.manual_seed_all(SEED)

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print("Device:", device)
print("Torch:", torch.__version__)
print("CUDA available:", torch.cuda.is_available())
if torch.cuda.is_available():
    print("GPU:", torch.cuda.get_device_name(0))


Device: cuda
Torch: 2.7.1+cu118
CUDA available: True
GPU: NVIDIA GeForce RTX 4060 Laptop GPU


2) Dataset + Transforms + Loaders + Class weights

In [11]:
# Train transform: daha güçlü augment (FER için işe yarıyor)
train_tf = transforms.Compose([
    transforms.Grayscale(num_output_channels=1),
    transforms.RandomResizedCrop(IMG_SIZE, scale=(0.75, 1.0)),
    transforms.RandomHorizontalFlip(p=0.5),
    transforms.RandomApply([transforms.ColorJitter(brightness=0.25, contrast=0.25)], p=0.5),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.5], std=[0.5]),
    transforms.RandomErasing(p=0.25, scale=(0.02, 0.12), ratio=(0.3, 3.3), value=0.0),
])

test_tf = transforms.Compose([
    transforms.Grayscale(num_output_channels=1),
    transforms.Resize((IMG_SIZE, IMG_SIZE)),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.5], std=[0.5]),
])

train_dir = os.path.join(DATASET_ROOT, "train")
test_dir  = os.path.join(DATASET_ROOT, "test")

train_ds = datasets.ImageFolder(train_dir, transform=train_tf)
test_ds  = datasets.ImageFolder(test_dir,  transform=test_tf)

classes = train_ds.classes
num_classes = len(classes)

print("Classes:", classes)
print("Train:", len(train_ds), "Test:", len(test_ds))

# class counts
y_train = np.array([train_ds.targets[i] for i in range(len(train_ds))], dtype=np.int64)
counts = np.bincount(y_train, minlength=num_classes).astype(np.float64)

# class weights (opsiyonel)
class_weights = (counts.sum() / (counts + 1e-9))
class_weights = class_weights / class_weights.mean()
print("Class weights (raw):", np.round(class_weights, 3))

# sampler
sampler = None
if USE_SAMPLER:
    w_per_sample = class_weights[y_train]
    sampler = WeightedRandomSampler(
        weights=torch.tensor(w_per_sample, dtype=torch.double),
        num_samples=len(w_per_sample),
        replacement=True
    )

train_loader = DataLoader(
    train_ds, batch_size=BATCH,
    shuffle=(sampler is None),
    sampler=sampler,
    num_workers=NUM_WORKERS,
    pin_memory=True,
    persistent_workers=(NUM_WORKERS > 0)
)

test_loader = DataLoader(
    test_ds, batch_size=BATCH,
    shuffle=False,
    num_workers=NUM_WORKERS,
    pin_memory=True,
    persistent_workers=(NUM_WORKERS > 0)
)


Classes: ['angry', 'disgust', 'fear', 'happy', 'neutral', 'sad', 'surprise']
Train: 28709 Test: 7178
Class weights (raw): [0.48  4.398 0.468 0.266 0.386 0.397 0.605]


3) Model(ResNet34) + Freeze/Unfreeze

In [12]:
def build_model(num_classes: int, backbone="resnet34", dropout=0.2):
    if backbone == "resnet18":
        m = models.resnet18(weights=models.ResNet18_Weights.DEFAULT)
        in_feats = m.fc.in_features
    elif backbone == "resnet34":
        m = models.resnet34(weights=models.ResNet34_Weights.DEFAULT)
        in_feats = m.fc.in_features
    else:
        raise ValueError("backbone must be resnet18 or resnet34")

    # 1 kanal input için conv1 değiştir
    old = m.conv1
    m.conv1 = nn.Conv2d(
        1, old.out_channels,
        kernel_size=old.kernel_size,
        stride=old.stride,
        padding=old.padding,
        bias=False
    )

    # pretrained RGB -> grayscale adapt: ağırlıkları ortalama ile kopyalanır
    with torch.no_grad():
        m.conv1.weight.copy_(old.weight.mean(dim=1, keepdim=True))

    # head
    m.fc = nn.Sequential(
        nn.Dropout(dropout),
        nn.Linear(in_feats, num_classes)
    )
    return m

def freeze_backbone(model):
    for name, p in model.named_parameters():
        if not name.startswith("fc."):
            p.requires_grad = False

def unfreeze_layer4(model):
    # resnet layer4 trainable
    for name, p in model.named_parameters():
        if name.startswith("layer4.") or name.startswith("fc."):
            p.requires_grad = True

def count_trainable_params(model):
    return sum(p.numel() for p in model.parameters() if p.requires_grad)


4) Karcı Optimizer(CNN için) + Momentum-Karcı

In [13]:
import torch

class KarciFANN(torch.optim.Optimizer):
    """
    KarcıFANN update (math aynı):
      ratio = (loss+eps) / (wref+eps)
      scale = ratio^(alpha-1)
      u     = scale * grad
      p     = p - lr * clamp(u)

    Notlar:
    - lr burada klasik LR gibi değil, "step_scale" gibi davranır.
    - wref: negatif ağırlık gelirse "önceki pozitif referansı koru" mantığı.
    """

    def __init__(self, params, alpha=1.30, clip=0.02, eps=1e-3,
                 rclip=(1e-3, 1e3), w_floor=1e-2, lr=1.0,
                 skip_on_nonfinite=True):
        defaults = dict(
            alpha=float(alpha),
            clip=None if clip is None else float(clip),
            eps=float(eps),
            rclip=(float(rclip[0]), float(rclip[1])),
            w_floor=float(w_floor),
            lr=float(lr),
            skip_on_nonfinite=bool(skip_on_nonfinite),
        )
        super().__init__(params, defaults)

        # state init: wref her param için pozitif tutulur
        for group in self.param_groups:
            eps = group["eps"]
            w_floor = group["w_floor"]
            for p in group["params"]:
                if p.requires_grad:
                    st = self.state[p]
                    st["wref"] = p.data.abs().clone().add_(eps).clamp_min_(w_floor)

    @torch.no_grad()
    def step(self, loss_scalar: float):
        # loss_scalar float olmalı (train loop'ta loss.item() veriyoruz)
        try:
            lossv = float(loss_scalar)
        except Exception:
            raise ValueError("KarcıFANN.step(loss_scalar) için loss_scalar float olmalı (örn: loss.item()).")

        for group in self.param_groups:
            alpha = group["alpha"]
            clipv = group["clip"]
            eps   = group["eps"]
            w_floor = group["w_floor"]
            lr = group["lr"]
            rlo, rhi = group["rclip"]
            pwr = alpha - 1.0
            skip = group["skip_on_nonfinite"]

            # loss NaN/inf ise: ya adım atlanır ya da sabitlenir
            if not (lossv == lossv and abs(lossv) != float("inf")):  # NaN/inf check (hızlı)
                if skip:
                    return None
                else:
                    lossv = 1.0  # fallback

            num = (lossv + eps)  # pozitif

            for p in group["params"]:
                if p.grad is None:
                    continue

                g = p.grad
                # grad NaN/inf ise temizle (çok önemli)
                if not torch.isfinite(g).all():
                    if skip:
                        # bu paramı güncellenmemeli
                        continue
                    g = torch.nan_to_num(g, nan=0.0, posinf=0.0, neginf=0.0)

                st = self.state[p]
                wref = st["wref"]

                # wref update: p.data pozitifse güncelleriz, değilse eski wref kalsın
                st["wref"] = torch.where(p.data > 0, p.data, wref)

                # wref_pos: kesin pozitif, çok küçükse w_floor'a çekeriz
                wref_pos = st["wref"].abs().clamp_min_(w_floor)

                # ratio kesin pozitif
                ratio = (num / (wref_pos + eps)).clamp_(min=rlo, max=rhi)

                # scale = ratio^(alpha-1)  (ratio>0 => NaN üretmez)
                scale = ratio.pow(pwr)

                # u = scale * grad
                u = scale * g

                # elementwise clip: patlamayı engeller
                if clipv is not None:
                    u = u.clamp_(min=-clipv, max=clipv)

                # param update
                p.data.add_(-lr * u)

        return None


class MomentumKarci(torch.optim.Optimizer):
    """
    Momentum-Karcı:
      u = ((loss+eps)/(wref+eps))^(alpha-1) * grad
      v = beta*v + u
      p = p - lr*v
    """

    def __init__(self, params, alpha=1.30, beta=0.90, clip=0.02, eps=1e-3,
                 rclip=(1e-3, 1e3), w_floor=1e-2, lr=1.0,
                 skip_on_nonfinite=True):
        defaults = dict(
            alpha=float(alpha),
            beta=float(beta),
            clip=None if clip is None else float(clip),
            eps=float(eps),
            rclip=(float(rclip[0]), float(rclip[1])),
            w_floor=float(w_floor),
            lr=float(lr),
            skip_on_nonfinite=bool(skip_on_nonfinite),
        )
        super().__init__(params, defaults)

        for group in self.param_groups:
            eps = group["eps"]
            w_floor = group["w_floor"]
            for p in group["params"]:
                if p.requires_grad:
                    st = self.state[p]
                    st["wref"] = p.data.abs().clone().add_(eps).clamp_min_(w_floor)
                    st["v"] = torch.zeros_like(p.data)

    @torch.no_grad()
    def step(self, loss_scalar: float):
        try:
            lossv = float(loss_scalar)
        except Exception:
            raise ValueError("MomentumKarci.step(loss_scalar) için loss_scalar float olmalı (örn: loss.item()).")

        for group in self.param_groups:
            alpha = group["alpha"]
            beta  = group["beta"]
            clipv = group["clip"]
            eps   = group["eps"]
            w_floor = group["w_floor"]
            lr = group["lr"]
            rlo, rhi = group["rclip"]
            pwr = alpha - 1.0
            skip = group["skip_on_nonfinite"]

            if not (lossv == lossv and abs(lossv) != float("inf")):
                if skip:
                    return None
                else:
                    lossv = 1.0

            num = (lossv + eps)

            for p in group["params"]:
                if p.grad is None:
                    continue

                g = p.grad
                if not torch.isfinite(g).all():
                    if skip:
                        continue
                    g = torch.nan_to_num(g, nan=0.0, posinf=0.0, neginf=0.0)

                st = self.state[p]

                st["wref"] = torch.where(p.data > 0, p.data, st["wref"])
                wref_pos = st["wref"].abs().clamp_min_(w_floor)

                ratio = (num / (wref_pos + eps)).clamp_(min=rlo, max=rhi)
                scale = ratio.pow(pwr)

                u = scale * g
                if clipv is not None:
                    u = u.clamp_(min=-clipv, max=clipv)

                st["v"].mul_(beta).add_(u)
                p.data.add_(-lr * st["v"])

        return None


5) Optimizer factory + Loss

In [14]:
def make_optimizer(opt_name, model, backbone_lr=3e-4, head_lr=3e-3, weight_decay=0.01):
    backbone_params = []
    head_params = []
    for name, p in model.named_parameters():
        if not p.requires_grad:
            continue
        if name.startswith("fc."):
            head_params.append(p)
        else:
            backbone_params.append(p)

    if opt_name == "adamw":
        return torch.optim.AdamW(
            [{"params": backbone_params, "lr": backbone_lr},
             {"params": head_params,     "lr": head_lr}],
            weight_decay=weight_decay
        )

    if opt_name == "sgd":
        return torch.optim.SGD(
            [{"params": backbone_params, "lr": backbone_lr},
             {"params": head_params,     "lr": head_lr}],
            momentum=0.9, nesterov=True, weight_decay=weight_decay
        )

    if opt_name == "karci":
        return KarciFANN(
            [{"params": backbone_params},
             {"params": head_params}],
            alpha=KARCI_ALPHA, clip=KARCI_CLIP, eps=KARCI_EPS
        )

    if opt_name == "mkarci":
        return MomentumKarci(
            [{"params": backbone_params},
             {"params": head_params}],
            alpha=KARCI_ALPHA, beta=0.85, clip=KARCI_CLIP, eps=KARCI_EPS
        )

    raise ValueError("opt_name must be one of: adamw, sgd, karci, mkarci")

# loss
cw = None
if USE_CLASS_WEIGHTS:
    cw = torch.tensor(class_weights, dtype=torch.float32, device=device)

criterion = nn.CrossEntropyLoss(weight=cw, label_smoothing=LABEL_SMOOTH)


6)Train/Eval(AMP doğru kullanımı + scheduler sırası düzeltildi)

In [15]:
import numpy as np
import torch
from sklearn.metrics import accuracy_score

# device (zaten tanımlıysa sorun değil)
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print("device:", device)

# Tek scaler (CPU'da otomatik disable)
scaler_amp = torch.amp.GradScaler(enabled=(device.type == "cuda"))

@torch.no_grad()
def evaluate(model, loader):
    model.eval()
    ys, ps = [], []
    for xb, yb in loader:
        xb = xb.to(device, non_blocking=True)
        yb = yb.to(device, non_blocking=True)

        with torch.autocast(device_type=device.type, enabled=(device.type == "cuda")):
            logits = model(xb)

        pred = torch.argmax(logits, dim=1)
        ys.append(yb.detach().cpu().numpy())
        ps.append(pred.detach().cpu().numpy())

    y = np.concatenate(ys)
    p = np.concatenate(ps)
    return float(accuracy_score(y, p)), y, p


def train_one_epoch(model, loader, optimizer, criterion,
                    scheduler=None, karci=False, grad_clip=1.0):
    model.train()
    total_loss = 0.0
    n = 0

    for xb, yb in loader:
        xb = xb.to(device, non_blocking=True)
        yb = yb.to(device, non_blocking=True)

        optimizer.zero_grad(set_to_none=True)

        # forward + loss (AMP)
        with torch.autocast(device_type=device.type, enabled=(device.type == "cuda")):
            logits = model(xb)
            loss = criterion(logits, yb)

        # loss NaN/Inf ise batch'i atlarız (Karcı NaN zincirini kesmek için çok önemli)
        if not torch.isfinite(loss):
            optimizer.zero_grad(set_to_none=True)
            continue

        # backward (scaled)
        scaler_amp.scale(loss).backward()

        # unscale -> clip doğru çalışsın
        scaler_amp.unscale_(optimizer)

        # gradient NaN/Inf kontrolü (özellikle Karcı'da patlama olursa)
        if grad_clip is not None:
            torch.nn.utils.clip_grad_norm_(model.parameters(), grad_clip)

        # optimizer step
        if karci:
            # Karcı optimizer step(loss_scalar) imzası
            optimizer.step(float(loss.detach().item()))
        else:
            scaler_amp.step(optimizer)

        scaler_amp.update()

        # OneCycle gibi scheduler'lar batch başına step ister
        if scheduler is not None:
            scheduler.step()

        bs = xb.size(0)
        total_loss += float(loss.detach().item()) * bs
        n += bs

    return total_loss / max(1, n)


device: cuda


7)Tek optimizer çalıştırma(her biri ayrı kayıt) + OneCycleLR fix(cycle_momentum)

In [16]:
import os, time, json
from sklearn.metrics import confusion_matrix

def train_one_optimizer(opt_name, backbone="resnet34"):
    run_dir = os.path.join(SAVE_DIR, opt_name)
    os.makedirs(run_dir, exist_ok=True)

    model = build_model(num_classes, backbone=backbone, dropout=0.2).to(device)
    freeze_backbone(model)

    optimizer = make_optimizer(opt_name, model)

    # OneCycleLR sadece adamw/sgd için (Karcı'da genelde kapalı daha stabil)
    steps_per_epoch = len(train_loader)
    use_scheduler = opt_name in ("adamw", "sgd")
    scheduler = None

    if use_scheduler:
        max_lrs = [pg.get("lr", 3e-4) for pg in optimizer.param_groups]
        scheduler = torch.optim.lr_scheduler.OneCycleLR(
            optimizer,
            max_lr=max_lrs,
            epochs=EPOCHS,
            steps_per_epoch=steps_per_epoch,
            pct_start=0.15,
            div_factor=10.0,
            final_div_factor=50.0,
            cycle_momentum=(opt_name == "sgd")
        )

    best_acc = -1.0
    best_ep = -1
    best_path = None
    did_unfreeze = False
    history = {"train_loss": [], "test_acc": []}

    for ep in range(1, EPOCHS + 1):

        # ---- unfreeze ----
        if (not did_unfreeze) and (ep == UNFREEZE_EPOCH):
            did_unfreeze = True
            print(f"[{opt_name.upper()}] Unfreeze layer4 at epoch {ep}")

            unfreeze_layer4(model)
            optimizer = make_optimizer(opt_name, model)

            # scheduler yeniden kurulur (optimizer değişti)
            if opt_name in ("adamw", "sgd"):
                max_lrs = [pg.get("lr", 3e-4) for pg in optimizer.param_groups]
                scheduler = torch.optim.lr_scheduler.OneCycleLR(
                    optimizer,
                    max_lr=max_lrs,
                    epochs=EPOCHS-ep+1,
                    steps_per_epoch=steps_per_epoch,
                    pct_start=0.15,
                    div_factor=10.0,
                    final_div_factor=50.0,
                    cycle_momentum=(opt_name == "sgd")
                )

            print("Trainable params:", count_trainable_params(model))

        t0 = time.time()
        karci_mode = opt_name in ("karci", "mkarci")

        tr_loss = train_one_epoch(
            model, train_loader, optimizer, criterion,
            scheduler=scheduler, karci=karci_mode, grad_clip=1.0
        )

        acc, y_true, y_pred = evaluate(model, test_loader)

        history["train_loss"].append(float(tr_loss))
        history["test_acc"].append(float(acc))

        if acc > best_acc:
            best_acc = float(acc)
            best_ep  = int(ep)
            best_path = os.path.join(run_dir, f"best_{opt_name}.pt")
            NORM_CFG = {
    "mode": "gray",
    "scale": "0_1",
    "mean": [0.5],
    "std":  [0.5],
    "channels": 1
}
            torch.save({
    "epoch": best_ep,
    "acc": best_acc,
    "model_state": model.state_dict(),
    "optimizer_name": opt_name,
    "classes": classes,
    "img_size": IMG_SIZE,
    "backbone": backbone,

    # >>> BUNLAR ÇOK KRİTİK:
    "norm": NORM_CFG,
    "input_channels": 1,
    "face_crop": True,   # webcam tarafında da yapacağız

    "history": history
}, best_path)


        dt = time.time() - t0

        lr_back = optimizer.param_groups[0].get("lr", None)
        lr_head = optimizer.param_groups[-1].get("lr", None)

        print(f"[{opt_name.upper()}] ep={ep:3d} loss={tr_loss:.4f} acc={acc:.3f} "
              f"| best={best_acc:.3f}@{best_ep} | lr(back)={lr_back} lr(head)={lr_head} ({dt:.1f}s)")

    # ---- BEST confusion ----
    ckpt = torch.load(best_path, map_location=device)
    model.load_state_dict(ckpt["model_state"])
    acc_best, y_true, y_pred = evaluate(model, test_loader)
    cm = confusion_matrix(y_true, y_pred)

    print("\nBest:", best_acc, "@epoch", best_ep, "path:", best_path)
    print(f"Confusion Matrix (BEST @{ckpt['epoch']} acc={acc_best:.4f}):\n", cm)

    meta = {
        "optimizer": opt_name,
        "epochs": int(EPOCHS),
        "unfreeze_epoch": int(UNFREEZE_EPOCH),
        "batch": int(BATCH),
        "img_size": int(IMG_SIZE),
        "use_sampler": bool(USE_SAMPLER),
        "use_class_weights": bool(USE_CLASS_WEIGHTS),
        "label_smoothing": float(LABEL_SMOOTH),
        "karci_alpha": float(KARCI_ALPHA) if opt_name in ("karci","mkarci") else None,
        "best_acc": float(best_acc),
        "best_epoch": int(best_ep),
        "classes": list(map(str, classes)),
        "best_path": best_path,
        "confusion_matrix": cm.tolist(),
    }

    with open(os.path.join(run_dir, f"{opt_name}.json"), "w", encoding="utf-8") as f:
        json.dump(meta, f, ensure_ascii=False, indent=2)

    print("Saved:", os.path.join(run_dir, f"{opt_name}.json"))
    return meta


8)Compare: AdamW + SGD + Karcı + Momentum-Karcı ve hepsini kaydederiz

In [ ]:
opts = ["adamw", "sgd", "karci", "mkarci"]  # istediğimizi çıkarıp ekleriz
results = []

for opt in opts:
    print("\n" + "="*90)
    print("RUN:", opt)
    print("="*90)
    meta = train_one_optimizer(opt_name=opt, backbone="resnet34")
    results.append(meta)

# overall best
best = max(results, key=lambda x: x["best_acc"])
with open(os.path.join(SAVE_DIR, "best_overall.json"), "w", encoding="utf-8") as f:
    json.dump(best, f, ensure_ascii=False, indent=2)

print("\nBEST OVERALL:", best["optimizer"], "acc=", best["best_acc"], "path=", best["best_path"])
print("Saved:", os.path.join(SAVE_DIR, "best_overall.json"))


9) Hepsini sırayla çalıştırırız (adamw/sgd/karci/mkarci)

In [35]:
#opts = ["adamw", "sgd", "karci", "mkarci"]  # istediğimizi çıkarıp ekleriz
opts = ["karci", "mkarci"]  # istediğimizi çıkarıp ekleriz
results = []

for opt in opts:
    print("\n" + "="*90)
    print("RUN:", opt)
    print("="*90)
    meta = train_one_optimizer(opt_name=opt, backbone="resnet34")
    results.append(meta)

# overall best
best = max(results, key=lambda x: x["best_acc"])
with open(os.path.join(SAVE_DIR, "best_overall.json"), "w", encoding="utf-8") as f:
    json.dump(best, f, ensure_ascii=False, indent=2)

print("\nBEST OVERALL:", best["optimizer"], "acc=", best["best_acc"], "path=", best["best_path"])
print("Saved:", os.path.join(SAVE_DIR, "best_overall.json"))



RUN: karci
[KARCI] ep=  1 loss=21.2777 acc=0.174 | best=0.174@1 | lr(back)=1.0 lr(head)=1.0 (26.7s)
[KARCI] ep=  2 loss=17.6863 acc=0.176 | best=0.176@2 | lr(back)=1.0 lr(head)=1.0 (26.6s)
[KARCI] Unfreeze layer4 at epoch 3
Trainable params: 13117959
[KARCI] ep=  3 loss=3.1683 acc=0.236 | best=0.236@3 | lr(back)=1.0 lr(head)=1.0 (31.5s)
[KARCI] ep=  4 loss=1.3917 acc=0.373 | best=0.373@4 | lr(back)=1.0 lr(head)=1.0 (31.4s)
[KARCI] ep=  5 loss=1.2269 acc=0.513 | best=0.513@5 | lr(back)=1.0 lr(head)=1.0 (31.6s)
[KARCI] ep=  6 loss=1.1485 acc=0.316 | best=0.513@5 | lr(back)=1.0 lr(head)=1.0 (31.3s)
[KARCI] ep=  7 loss=1.0884 acc=0.534 | best=0.534@7 | lr(back)=1.0 lr(head)=1.0 (31.5s)
[KARCI] ep=  8 loss=1.0427 acc=0.498 | best=0.534@7 | lr(back)=1.0 lr(head)=1.0 (31.3s)
[KARCI] ep=  9 loss=1.0115 acc=0.551 | best=0.551@9 | lr(back)=1.0 lr(head)=1.0 (31.5s)
[KARCI] ep= 10 loss=0.9867 acc=0.554 | best=0.554@10 | lr(back)=1.0 lr(head)=1.0 (31.5s)
[KARCI] ep= 11 loss=0.9618 acc=0.565 | best

In [17]:
import cv2, torch, numpy as np
import torch.nn.functional as F

FACE_CASCADE = cv2.CascadeClassifier(cv2.data.haarcascades + "haarcascade_frontalface_default.xml")

def load_checkpoint(pt_path):
    ckpt = torch.load(pt_path, map_location=device)
    model = build_model(num_classes, backbone=ckpt.get("backbone","resnet34"), dropout=0.0).to(device)
    model.load_state_dict(ckpt["model_state"])
    model.eval()
    return model, ckpt

def get_face_crop(frame_bgr):
    gray = cv2.cvtColor(frame_bgr, cv2.COLOR_BGR2GRAY)
    faces = FACE_CASCADE.detectMultiScale(gray, scaleFactor=1.1, minNeighbors=5, minSize=(60, 60))
    if len(faces) == 0:
        return None
    # en büyük yüzü seçelim
    x,y,w,h = sorted(faces, key=lambda b: b[2]*b[3], reverse=True)[0]
    pad = int(0.15 * max(w,h))
    x0 = max(0, x-pad); y0 = max(0, y-pad)
    x1 = min(frame_bgr.shape[1], x+w+pad); y1 = min(frame_bgr.shape[0], y+h+pad)
    return frame_bgr[y0:y1, x0:x1]

def preprocess_frame(frame_bgr, img_size, norm_cfg=None):
    # yüz crop varsa onu kullanalım
    face = get_face_crop(frame_bgr)
    if face is None:
        face = frame_bgr  # yüz bulunamazsa full frame (ama bu daha kötü olur)

    gray = cv2.cvtColor(face, cv2.COLOR_BGR2GRAY)
    img  = cv2.resize(gray, (img_size, img_size), interpolation=cv2.INTER_AREA)

    x = torch.from_numpy(img).float() / 255.0  # [H,W] 0..1

    # norm cfg
    if norm_cfg is None:
        mean, std = 0.5, 0.5
    else:
        mean = float(norm_cfg.get("mean", [0.5])[0])
        std  = float(norm_cfg.get("std",  [0.5])[0])

    x = (x - mean) / (std + 1e-8)

    x = x.unsqueeze(0).unsqueeze(0)  # [1,1,H,W]
    return x.to(device)


9) Webcam demo (tek model)

In [18]:
import cv2

def load_checkpoint(pt_path):
    ckpt = torch.load(pt_path, map_location=device)
    model = build_model(num_classes, backbone=ckpt.get("backbone","resnet34"), dropout=0.0).to(device)
    model.load_state_dict(ckpt["model_state"])
    model.eval()
    return model, ckpt

# aynı normalize ile preprocess
def preprocess_frame_bgr(frame_bgr):
    gray = cv2.cvtColor(frame_bgr, cv2.COLOR_BGR2GRAY)
    img = cv2.resize(gray, (IMG_SIZE, IMG_SIZE), interpolation=cv2.INTER_AREA)
    x = torch.from_numpy(img).float() / 255.0
    x = (x - 0.5) / 0.5
    x = x.unsqueeze(0).unsqueeze(0)  # [1,1,H,W]
    return x.to(device)

def webcam_run(pt_path, cam_id=0):
    model, ckpt = load_checkpoint(pt_path)
    cls = ckpt["classes"]

    cap = cv2.VideoCapture(cam_id)
    if not cap.isOpened():
        raise RuntimeError("Webcam açılamadı. cam_id değiştir: 0/1/2 deneyebilirsin.")

    print("ESC ile çıkış.")
    while True:
        ok, frame = cap.read()
        if not ok:
            break

        x = preprocess_frame_bgr(frame)
        with torch.no_grad():
            logits = model(x)
            prob = torch.softmax(logits, dim=1)[0].detach().cpu().numpy()
            pred = int(prob.argmax())

        text = f"{cls[pred]} ({prob[pred]*100:.1f}%)"
        cv2.putText(frame, text, (20, 40), cv2.FONT_HERSHEY_SIMPLEX, 1.0, (0,255,0), 2)

        cv2.imshow("FER live", frame)
        k = cv2.waitKey(1) & 0xFF
        if k == 27:  # ESC
            break

    cap.release()
    cv2.destroyAllWindows()

webcam_run(r".\runs_cnn\adamw\best_adamw.pt")


ESC ile çıkış.


10) İki modeli aynı anda kıyas (A/B)

In [19]:
def webcam_compare(ptA, ptB, cam_id=0, conf_thr=0.35):
    modelA, ckptA = load_checkpoint(ptA)
    modelB, ckptB = load_checkpoint(ptB)

    clsA = ckptA.get("classes", None)
    clsB = ckptB.get("classes", None)

    img_sizeA = int(ckptA.get("img_size", 224))
    img_sizeB = int(ckptB.get("img_size", 224))

    normA = ckptA.get("norm", None)
    normB = ckptB.get("norm", None)

    cap = cv2.VideoCapture(cam_id)
    if not cap.isOpened():
        raise RuntimeError("Webcam açılamadı. cam_id=0/1 deneyebilirsin.")

    print("ESC çıkış.")
    while True:
        ok, frame = cap.read()
        if not ok:
            break

        xA = preprocess_frame(frame, img_sizeA, normA)
        xB = preprocess_frame(frame, img_sizeB, normB)

        with torch.no_grad():
            pA = torch.softmax(modelA(xA), dim=1)[0]
            pB = torch.softmax(modelB(xB), dim=1)[0]

        a = int(torch.argmax(pA)); b = int(torch.argmax(pB))
        ca = float(pA[a]); cb = float(pB[b])

        # düşük güven: “uncertain” yazalım (sad’e çakılmayı azaltır)
        la = "uncertain" if ca < conf_thr else (clsA[a] if clsA else str(a))
        lb = "uncertain" if cb < conf_thr else (clsB[b] if clsB else str(b))

        tA = f"A:{ckptA.get('optimizer_name','?')} {la} {ca*100:.1f}%"
        tB = f"B:{ckptB.get('optimizer_name','?')} {lb} {cb*100:.1f}%"

        cv2.putText(frame, tA, (20, 40), cv2.FONT_HERSHEY_SIMPLEX, 0.85, (255,255,0), 2)
        cv2.putText(frame, tB, (20, 80), cv2.FONT_HERSHEY_SIMPLEX, 0.85, (0,255,255), 2)

        cv2.imshow("FER live A/B", frame)
        k = cv2.waitKey(1) & 0xFF
        if k == 27:  # ESC
            break

    cap.release()
    cv2.destroyAllWindows()
# örnek:
webcam_compare(r".\runs_cnn\sgd\best_sgd.pt", r".\runs_cnn\karci\best_karci.pt")


ESC çıkış.


11) Webcam ile "seçtiğim.pt" canlı kıyas (iki modeli aynı anda)

In [20]:
import os
import cv2
import numpy as np
import torch

def load_checkpoint(pt_path):
    ckpt = torch.load(pt_path, map_location=device)

    # backbone bilgisi ckpt içinde yoksa varsayılan
    backbone = ckpt.get("backbone", "resnet34")

    # bazı ckpt'lerde num_classes yok, o yüzden classes üzerinden çıkaralım
    if "classes" in ckpt and ckpt["classes"] is not None:
        ncls = len(ckpt["classes"])
    else:
        ncls = num_classes  # senin global num_classes'ın varsa

    model = build_model(ncls, backbone=backbone, dropout=0.0).to(device)
    model.load_state_dict(ckpt["model_state"])
    model.eval()
    return model, ckpt

def preprocess_frame_bgr(frame_bgr):
    gray = cv2.cvtColor(frame_bgr, cv2.COLOR_BGR2GRAY)
    img  = cv2.resize(gray, (IMG_SIZE, IMG_SIZE), interpolation=cv2.INTER_AREA)
    x = torch.from_numpy(img).float() / 255.0
    x = (x - 0.5) / 0.5
    x = x.unsqueeze(0).unsqueeze(0)  # [1,1,H,W]
    return x.to(device)

def _model_name(ckpt, path):
    # ckpt içinde farklı isimlendirmeler olabiliyor
    if "optimizer_name" in ckpt and ckpt["optimizer_name"]:
        return str(ckpt["optimizer_name"]).upper()
    if "optimizer" in ckpt and ckpt["optimizer"]:
        return str(ckpt["optimizer"]).upper()
    # yoksa dosya adından
    return os.path.splitext(os.path.basename(path))[0].upper()

def webcam_compare(pt_path_A, pt_path_B, cam_id=0):
    modelA, ckptA = load_checkpoint(pt_path_A)
    modelB, ckptB = load_checkpoint(pt_path_B)

    # classes güvenli çekelim
    clsA = ckptA.get("classes", None)
    clsB = ckptB.get("classes", None)

    # sınıflar yoksa fallback
    if clsA is None and clsB is None:
        cls = [str(i) for i in range(num_classes)]
    elif clsA is not None and clsB is None:
        cls = clsA
    elif clsA is None and clsB is not None:
        cls = clsB
    else:
        # ikisi de var, uyuşuyor mu bakalım
        if list(clsA) != list(clsB):
            print("UYARI: A ve B sınıf listeleri farklı! A sınıfları baz alınacak.")
        cls = clsA

    nameA = _model_name(ckptA, pt_path_A)
    nameB = _model_name(ckptB, pt_path_B)

    cap = cv2.VideoCapture(cam_id)
    if not cap.isOpened():
        raise RuntimeError("Webcam açılamadı. cam_id=0/1/2 dene, ya da başka uygulama kullanıyor olabilir.")

    print("ESC ile çıkış.")

    while True:
        ok, frame = cap.read()
        if not ok:
            break

        x = preprocess_frame_bgr(frame)

        with torch.no_grad():
            pA = torch.softmax(modelA(x), dim=1)[0].detach().cpu().numpy()
            pB = torch.softmax(modelB(x), dim=1)[0].detach().cpu().numpy()

        a = int(np.argmax(pA))
        b = int(np.argmax(pB))

        tA = f"A:{nameA}  {cls[a]}  {pA[a]*100:.1f}%"
        tB = f"B:{nameB}  {cls[b]}  {pB[b]*100:.1f}%"

        # yazılar
        cv2.putText(frame, tA, (20, 40), cv2.FONT_HERSHEY_SIMPLEX, 0.85, (255,255,0), 2)
        cv2.putText(frame, tB, (20, 80), cv2.FONT_HERSHEY_SIMPLEX, 0.85, (0,255,255), 2)

        cv2.imshow("FER live A/B", frame)
        k = cv2.waitKey(1) & 0xFF
        if k == 27:  # ESC
            break

    cap.release()
    cv2.destroyAllWindows()

# örnek:
webcam_compare(r".\runs_cnn\mkarci\best_mkarci.pt", r".\runs_cnn\karci\best_karci.pt")


ESC ile çıkış.


In [50]:
ckpt = torch.load(r".\runs_cnn\karci\best_karci.pt", map_location="cpu")
print("img_size:", ckpt.get("img_size"))
print("norm:", ckpt.get("norm"))
print("backbone:", ckpt.get("backbone"))


img_size: 224
norm: None
backbone: resnet34
